In [28]:
# https://stackoverflow.com/questions/66116155/how-to-tell-pytorch-which-cuda-version-to-take
# https://github.com/pytorch/pytorch/issues/75992

# Previous Versions of Pytorch avaiable here: https://pytorch.org/get-started/previous-versions/
#! pip install --upgrade torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
#! pip install --upgrade torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0
#! pip install --upgrade torch==1.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 #<--This version works for CUDA 11.4



In [187]:
import pandas as pd
import os
import time
import numpy as np
import torch

In [2]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

In [3]:
torch.__version__
#'1.12.0+cu111'
#'1.9.0+cu111' This is the version by default


'1.12.1+cu113'

In [4]:
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('CPU')
physical_devices = tf.config.list_physical_devices('GPU')

In [5]:
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


device(type='cuda', index=0)

In [7]:
#test
x = torch.rand(5, 3, 
               device=device#'cpu'
               )
print(x)

tensor([[0.6204, 0.7299, 0.2628],
        [0.4466, 0.2726, 0.8782],
        [0.6232, 0.7642, 0.6538],
        [0.8070, 0.6778, 0.2704],
        [0.5194, 0.5688, 0.9396]], device='cuda:0')


In [8]:
x.get_device()

0

In [156]:
#! conda install pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch
#! pip3 install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html
#! pip install tensorflow
! nvidia-smi

Wed Feb 15 23:21:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   30C    P0    35W / 250W |  40416MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model @danaderp
>

In [298]:
#Available Datasets
# Case1: codesearch_tesbed_EleutherAI-gpt-neo-125M_10000 for the model 'EleutherAI/gpt-neo-125M' 
def params(): 
    return {
            'big_table_path' : '../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-125M_10000.csv',
            'hf_model' : 'EleutherAI/gpt-neo-125M',
            'model_name': 'EleutherAI-gpt-neo-125M_10000_',
            'callbacks' : '../data/callbacks',
            'batch': 10 
}

In [299]:
#pwd
parameters = params()
parameters['big_table_path']

'../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-125M_10000.csv'

# Data Upload

In [31]:
data_pd = pd.read_csv( 
                      parameters['big_table_path'] , 
                      index_col=0
            )

In [32]:
data_pd.describe()

,model_total_input_ids
count,10000.000000
mean,411.430700
std,358.223398
min,47.000000
25%,168.000000
50%,286.000000
75%,526.000000
max,2044.000000


In [33]:
data_pd.head(5)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def _sample_field(self, sample):\n """"""R...","[('def', 'def', 'function_definition'), ('_sam...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 39873, 62, 3245, 7, 944, 11, 6291...",154
1,"def create_cloud_service(self, cloud_service_i...","[('def', 'def', 'function_definition'), ('crea...","[(4299, 'def', 'function_definition'), (2251, ...","[4299, 2251, 62, 17721, 62, 15271, 7, 944, 11,...",459
2,"def clean_dict(dict):\n """"""Remove all keys ...","[('def', 'def', 'function_definition'), ('clea...","[(4299, 'def', 'function_definition'), (3424, ...","[4299, 3424, 62, 11600, 7, 11600, 2599, 198, 2...",151
3,"def get_pid_from_file(self):\n """"""Get t...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 35317, 62, 6738, 62, 7753, 7, ...",338
4,"async def get_artist(self, spotify_id: str) ->...","[('async', 'async', 'function_definition'), ('...","[(292, 'async', 'function_definition'), (13361...","[292, 13361, 825, 651, 62, 49016, 7, 944, 11, ...",185


In [37]:
data_pd['model_tokenizer_concepts'][0]

"[(4299, 'def', 'function_definition'), (4808, 'identifier', 'function_definition'), (39873, 'identifier', 'function_definition'), (62, 'identifier', 'function_definition'), (3245, 'identifier', 'function_definition'), (7, '(', 'parameters'), (944, 'identifier', 'parameters'), (11, ',', 'parameters'), (6291, 'identifier', 'parameters'), (2599, ')', 'parameters'), (198, 'ERROR', 'ERROR'), (220, 'ERROR', 'ERROR'), (220, 'ERROR', 'ERROR'), (220, 'ERROR', 'ERROR'), (220, 'ERROR', 'ERROR'), (220, 'ERROR', 'ERROR'), (220, 'ERROR', 'ERROR'), (220, 'ERROR', 'ERROR'), (37227, 'string', 'expression_statement'), (35561, 'string', 'expression_statement'), (4731, 'string', 'expression_statement'), (10552, 'string', 'expression_statement'), (286, 'string', 'expression_statement'), (6291, 'string', 'expression_statement'), (12, 'string', 'expression_statement'), (18982, 'string', 'expression_statement'), (3815, 'string', 'expression_statement'), (13, 'string', 'expression_statement'), (628, 'string',

In [35]:
data_pd.ast_concepts.values[0][2]

"'"

In [38]:
data_pd.whole_func_string.values[0]

'def _sample_field(self, sample):\n        """Returns string representation of sample-format values.\n\n        Raises:\n            KeyError: if requested sample is not defined.\n        """\n        tag_values = self.sample_tag_values[sample].values()\n        if tag_values:\n            return ":".join(tag_values)\n        else:\n            return "."'

# Model Upload

In [40]:
#! pip install transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [22]:
#pip list
#! pip install git+https://github.com/huggingface/transfomers.git


In [44]:
#tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

# This is for 'EleutherAI/gpt-neo-125M'
tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
generator = pipeline(
    'text-generation', 
    model= parameters['hf_model'] )


In [45]:
#TEST: Example 1: generation
generator(
    "EleutherAI has", 
    do_sample=True, 
    max_new_tokens=20, 
    pad_token_id = tokenizer.eos_token_id
    )

[{'generated_text': 'EleutherAI has become the first machine that makes it possible to simulate the life cycle of a human-sized humanoid living'}]

In [46]:
#TEST: Example 2: generation

generator( 
    data_pd.whole_func_string.values[0][:100], #<-- Code data
    do_sample=True, 
    max_new_tokens=20,
    pad_token_id = tokenizer.eos_token_id 
    )


[{'generated_text': 'def _sample_field(self, sample):\n        """Returns string representation of sample-format values.\n\n        :param sample: A value representing the sample format used by this'}]

In [47]:
#generator2 = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')


In [48]:
#generator2("for(int i= ", do_sample=True, max_new_tokens=50)

In [49]:
#generator2( data_pd.whole_func_string.values[0][:100], do_sample=True, max_new_tokens=50 )

# Extracting Logits From a Given Model

In [50]:
#This code works for GPT-Neo
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained( parameters['hf_model'] )
tokenizer = GPT2Tokenizer.from_pretrained( parameters['hf_model'] )

In [51]:
tokenizer.get_vocab()

{'!': 0,
 '"': 1,
 '#': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '<': 27,
 '=': 28,
 '>': 29,
 '?': 30,
 '@': 31,
 'A': 32,
 'B': 33,
 'C': 34,
 'D': 35,
 'E': 36,
 'F': 37,
 'G': 38,
 'H': 39,
 'I': 40,
 'J': 41,
 'K': 42,
 'L': 43,
 'M': 44,
 'N': 45,
 'O': 46,
 'P': 47,
 'Q': 48,
 'R': 49,
 'S': 50,
 'T': 51,
 'U': 52,
 'V': 53,
 'W': 54,
 'X': 55,
 'Y': 56,
 'Z': 57,
 '[': 58,
 '\\': 59,
 ']': 60,
 '^': 61,
 '_': 62,
 '`': 63,
 'a': 64,
 'b': 65,
 'c': 66,
 'd': 67,
 'e': 68,
 'f': 69,
 'g': 70,
 'h': 71,
 'i': 72,
 'j': 73,
 'k': 74,
 'l': 75,
 'm': 76,
 'n': 77,
 'o': 78,
 'p': 79,
 'q': 80,
 'r': 81,
 's': 82,
 't': 83,
 'u': 84,
 'v': 85,
 'w': 86,
 'x': 87,
 'y': 88,
 'z': 89,
 '{': 90,
 '|': 91,
 '}': 92,
 '~': 93,
 '¡': 94,
 '¢': 95,
 '£': 96,
 '¤': 97,
 '¥': 98,
 '¦': 99,
 '§': 100

In [52]:
len( tokenizer.get_vocab() ) #todo an assert

50257

In [53]:
#prompts = data_pd.whole_func_string.values[:2]
prompts = data_pd.whole_func_string.values

In [54]:
len(prompts)

10000

In [55]:
prompts[:2]

array(['def _sample_field(self, sample):\n        """Returns string representation of sample-format values.\n\n        Raises:\n            KeyError: if requested sample is not defined.\n        """\n        tag_values = self.sample_tag_values[sample].values()\n        if tag_values:\n            return ":".join(tag_values)\n        else:\n            return "."',
       "def create_cloud_service(self, cloud_service_id, label, description, geo_region):\n        '''\n        The Create Cloud Service request creates a new cloud service. When job\n        collections are created, they are hosted within a cloud service.\n        A cloud service groups job collections together in a given region.\n        Once a cloud service has been created, job collections can then be\n        created and contained within it.\n\n        cloud_service_id:\n            The cloud service id\n        label:\n            The name of the cloud service.\n        description:\n            The description of the c

In [142]:
#Making space for the tensors
input_ids_list = tokenizer.batch_encode_plus( prompts ) #<-- Do not return as a Tensor
#input_ids_list = tokenizer.batch_encode_plus( prompts , return_tensors="pt") #<-- "pt" returns as a Tensor

In [144]:
#Casting Integers to Tensor Integers. Make sure the tesor is created in a device
#We ignored the parameter attention_mask since we are not using masking here [https://huggingface.co/transformers/v4.10.1/glossary.html#attention-mask]

#input_ids_list = [torch.Tensor( np.array( input_ids ) ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
#input_ids_list = [ input_ids for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
input_ids_list = [torch.tensor(  input_ids, dtype = torch.int, device=device ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]

In [137]:
#It should be same size
assert len(input_ids_list) == len(prompts)

In [117]:
data_pd.model_input_ids.values[0]

'[4299, 4808, 39873, 62, 3245, 7, 944, 11, 6291, 2599, 198, 220, 220, 220, 220, 220, 220, 220, 37227, 35561, 4731, 10552, 286, 6291, 12, 18982, 3815, 13, 628, 220, 220, 220, 220, 220, 220, 220, 7567, 2696, 25, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 7383, 12331, 25, 611, 9167, 6291, 318, 407, 5447, 13, 198, 220, 220, 220, 220, 220, 220, 220, 37227, 198, 220, 220, 220, 220, 220, 220, 220, 7621, 62, 27160, 796, 2116, 13, 39873, 62, 12985, 62, 27160, 58, 39873, 4083, 27160, 3419, 198, 220, 220, 220, 220, 220, 220, 220, 611, 7621, 62, 27160, 25, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 1441, 366, 25, 1911, 22179, 7, 12985, 62, 27160, 8, 198, 220, 220, 220, 220, 220, 220, 220, 2073, 25, 198, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 220, 1441, 366, 526]'

In [138]:
input_ids_list[0]

tensor([ 4299,  4808, 39873,    62,  3245,     7,   944,    11,  6291,  2599,
          198,   220,   220,   220,   220,   220,   220,   220, 37227, 35561,
         4731, 10552,   286,  6291,    12, 18982,  3815,    13,   628,   220,
          220,   220,   220,   220,   220,   220,  7567,  2696,    25,   198,
          220,   220,   220,   220,   220,   220,   220,   220,   220,   220,
          220,  7383, 12331,    25,   611,  9167,  6291,   318,   407,  5447,
           13,   198,   220,   220,   220,   220,   220,   220,   220, 37227,
          198,   220,   220,   220,   220,   220,   220,   220,  7621,    62,
        27160,   796,  2116,    13, 39873,    62, 12985,    62, 27160,    58,
        39873,  4083, 27160,  3419,   198,   220,   220,   220,   220,   220,
          220,   220,   611,  7621,    62, 27160,    25,   198,   220,   220,
          220,   220,   220,   220,   220,   220,   220,   220,   220,  1441,
          366,    25,  1911, 22179,     7, 12985,    62, 27160, 

In [145]:
tokenizer.decode( input_ids_list[0] ) #Decoding IDs

'def _sample_field(self, sample):\n        """Returns string representation of sample-format values.\n\n        Raises:\n            KeyError: if requested sample is not defined.\n        """\n        tag_values = self.sample_tag_values[sample].values()\n        if tag_values:\n            return ":".join(tag_values)\n        else:\n            return "."'

In [310]:
#model.to( 'cpu' ) #WARNING, 
model.to( device ) #WARNING, Verify the device before assigning to memory

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_features=3

In [311]:
parameters

{'big_table_path': '../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-125M_10000.csv',
 'hf_model': 'EleutherAI/gpt-neo-125M',
 'model_name': 'EleutherAI-gpt-neo-125M_10000_',
 'callbacks': '../data/callbacks',
 'batch': 10}

In [309]:
def logit_extractor(batch, input):
    """
    Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)"
    logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
    The expression i.type(torch.LongTensor).to(device) is for casting labels for the loss
    """
    #Output is in CausalLMOutputWithPast

    for idx, n in enumerate( range( 0, len(input), batch) ):
        output = [ model( 
            input_ids = i, 
            labels = i.type(torch.LongTensor).to(device) 
            ) for i in input[n:n+batch] ] #Labels must be provided to compute loss
    
        output_logits = [ o.logits.detach().to('cpu').numpy() for o in output ]  #Logits Extraction
        output_loss = np.array([ o.loss.detach().to('cpu').numpy() for o in output ])  #Language modeling loss (for next-token prediction).

        #Saving Callbacks

        for jdx, o_logits in enumerate( output_logits ):
            np.save( parameters['callbacks']+ '/'+parameters['model_name']  + f'_logits_tensor[{jdx+n}]_batch[{idx}].npy', o_logits)
        np.save( parameters['callbacks']+ '/'+parameters['model_name']+f'_loss_batch[{idx}].npy', output_loss)
        print(f"Batch [{idx}] Completed")

    pass

In [302]:
logit_extractor(batch =2, input= input_ids_list[:2]) #<---- [WARNING TIME AND MEMORY CONSUMING]

Batch [0] Completed


In [288]:
output_logits = np.load('../data/callbacks/logits_tensor[0]_batch[0].npy')

In [290]:
assert output_logits.shape[0] == len(input_ids_list[0])

In [291]:
output_loss = np.load('../data/callbacks/loss_batch[0].npy')

In [308]:
output_loss

array([1.0893235, 1.0868526], dtype=float32)

In [305]:
## ACTUAL EXPERIMENT
## TIME AND MEMORY CONSUMING
logit_extractor(batch = 2, input= input_ids_list)

Batch [0] Completed
Batch [1] Completed
Batch [2] Completed
Batch [3] Completed
Batch [4] Completed
Batch [5] Completed
Batch [6] Completed
Batch [7] Completed


RuntimeError: CUDA out of memory. Tried to allocate 22.00 MiB (GPU 0; 39.59 GiB total capacity; 37.68 GiB already allocated; 22.19 MiB free; 38.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Softmax Normalization and Data Engineering

In [162]:
#Based on the response here: https://stackoverflow.com/questions/62852940/how-to-get-immediate-next-word-probability-using-gpt2-model
# next_token_logits = output[0][:, -1, :] #last token
output_logits[0].shape #The loggits contains the information of the next token prediction given the prevous windows w_{<t}

torch.Size([154, 50257])

In [163]:
output_logits[0][0]

tensor([ -5.6060,  -2.4399,  -6.9558,  ..., -11.8578, -14.6072,  -6.0800],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [165]:
sum( output_logits[0][0] ) #non-normalized tokens summup different than 1

tensor(-553426.4375, device='cuda:0', grad_fn=<AddBackward0>)

In [49]:
soft = torch.nn.Softmax( dim = 0 ) #Flattening normalization
next_token_distribution = soft( output_logits[1][0] ) #Flattening normalization

In [50]:
next_token_distribution.shape

torch.Size([50257])

## Batch Softmax Distribution

In [51]:
output_logits

[tensor([[ -5.6059,  -2.4399,  -6.9557,  ..., -11.8578, -14.6071,  -6.0799],
         [ -6.2990,  -3.9610,  -6.1833,  ..., -10.6406,  -8.8349,  -6.3452],
         [ -4.8018,  -4.1277,  -4.1619,  ...,  -8.1974,  -7.6979,  -6.0528],
         ...,
         [-11.9002, -11.7730,  -8.0985,  ..., -15.8561, -17.3783, -11.6446],
         [ -7.4098,  -9.9632, -12.2038,  ..., -18.2707, -18.1764, -10.3717],
         [-10.0745,  -9.8232, -12.0393,  ..., -25.0120, -21.1442,  -8.6445]],
        grad_fn=<MmBackward0>),
 tensor([[ -5.6059,  -2.4399,  -6.9557,  ..., -11.8578, -14.6071,  -6.0799],
         [ -3.2613,  -3.4885,  -7.5667,  ..., -14.2281, -14.1252,  -6.1792],
         [ -4.1326,  -5.6764,  -5.4887,  ..., -12.1207, -11.4404,  -7.5004],
         ...,
         [-25.2886, -25.3481, -24.2219,  ..., -32.4984, -33.6202, -15.5898],
         [-23.1931, -22.4658, -20.3607,  ..., -23.5468, -28.6755, -14.5011],
         [-16.4041, -14.7310, -12.8584,  ..., -31.2271, -25.5518,  -5.8683]],
        grad_f

In [52]:
next_token_distributions = [ [soft( logits ) for logits in prompt_tesor]  for prompt_tesor in output_logits ]

In [53]:
sum( next_token_distributions[0][0] ) #tst

tensor(1.0000, grad_fn=<AddBackward0>)

In [54]:
def topk_tuple( vocab_token, largest):
    token_position_dict = list( tokenizer.get_vocab().keys() )
    topk = vocab_token.topk( k=1 , largest=largest ) #TODO K number of elements can be extended
    return ( token_position_dict[topk.indices], topk.values)

### MAX Case

In [55]:
#maximum case
#return A namedtuple of (values, indices)
#If Largest True then search for the max case
max_logit_token_prompt = [[ topk_tuple(vocab_token=vocab_token, largest=True) for vocab_token in prompt ] for prompt in next_token_distributions ]
max_logit_token_prompt

[[('ers', tensor([0.0308], grad_fn=<TopkBackward0>)),
  ('_', tensor([0.2419], grad_fn=<TopkBackward0>)),
  ('name', tensor([0.0194], grad_fn=<TopkBackward0>)),
  ('_', tensor([0.4209], grad_fn=<TopkBackward0>)),
  ('node', tensor([0.2215], grad_fn=<TopkBackward0>)),
  (',', tensor([0.4757], grad_fn=<TopkBackward0>)),
  ('Ġnode', tensor([0.1696], grad_fn=<TopkBackward0>)),
  (',', tensor([0.3714], grad_fn=<TopkBackward0>)),
  ('Ċ', tensor([0.9505], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9688], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9873], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9983], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.4134], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9901], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9468], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9975], grad_fn=<TopkBackward0>)),
  ('Ġ"""', tensor([0.1968], grad_fn=<TopkBackward0>)),
  ('Ċ', tensor([0.4083], grad_fn=<TopkBackward0>)),
  ('Ġ', tensor([0.9993], grad_fn=<TopkBackward0>)

In [56]:
data_pd['max_prob_case'] = max_logit_token_prompt

In [57]:
data_pd.head(1)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case
0,"def get_node(self, label):\n """"""\n ...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 17440, 7, 944, 11, 6167, 2599,...",115,"[(ers, [tensor(0.0308, grad_fn=<UnbindBackward..."


### MIN Case

In [58]:
#minimum case
min_logit_token_prompt = [[ topk_tuple(vocab_token=vocab_token, largest=False) for vocab_token in prompt ] for prompt in next_token_distributions ]

In [59]:
data_pd['min_prob_case'] = min_logit_token_prompt
data_pd.head(1)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case
0,"def get_node(self, label):\n """"""\n ...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 17440, 7, 944, 11, 6167, 2599,...",115,"[(ers, [tensor(0.0308, grad_fn=<UnbindBackward...","[(anwhile, [tensor(1.0384e-16, grad_fn=<Unbind..."


### Actual Token Case

In [60]:
#actual token case
actual_logit_token_prompt = []
token_position_dict = list( tokenizer.get_vocab().keys() )
for p,prompt in enumerate( input_ids_list ):
    temp_prompt = []
    for c,code_token_position in enumerate( prompt[1:] ): #Eliminate the first token prediction since we do not use it
        temp_prompt.append( 
                ( token_position_dict[int(code_token_position)],next_token_distributions[ p ][ c ][int(code_token_position)] )        
            )
    actual_logit_token_prompt.append( temp_prompt )

In [61]:
data_pd['actual_prob_case'] = actual_logit_token_prompt
data_pd.head(1)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case
0,"def get_node(self, label):\n """"""\n ...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 17440, 7, 944, 11, 6167, 2599,...",115,"[(ers, [tensor(0.0308, grad_fn=<UnbindBackward...","[(anwhile, [tensor(1.0384e-16, grad_fn=<Unbind...","[(Ġget, tensor(0.0024, grad_fn=<SelectBackward..."


In [90]:
dataframe_to_save = data_pd.copy()
dataframe_to_save['max_prob_case'] = dataframe_to_save['max_prob_case'].map(lambda max_prob: [(value[0], value[1].item()) for value in max_prob ])
dataframe_to_save['min_prob_case'] = dataframe_to_save['min_prob_case'].map(lambda min_prob: [(value[0], value[1].item()) for value in min_prob ])
dataframe_to_save['actual_prob_case'] = dataframe_to_save['actual_prob_case'].map(lambda actual_prob: [(value[0], value[1].item()) for value in actual_prob ])
dataframe_to_save.head()
#dataframe_to_save.iloc[0]['actual_prob_case'][0]

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case
0,"def get_node(self, label):\n """"""\n ...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 17440, 7, 944, 11, 6167, 2599,...",115,"[(ers, 0.03083181567490101), (_, 0.24190878868...","[(anwhile, 1.03836617568492e-16), (ousy, 3.281...","[(Ġget, 0.0024285861290991306), (_, 0.24190878..."
1,"def execute_pipeline(pipeline, environment_dic...","[('def', 'def', 'function_definition'), ('exec...","[(4299, 'def', 'function_definition'), (12260,...","[4299, 12260, 62, 79, 541, 4470, 7, 79, 541, 4...",492,"[(ers, 0.03083181567490101), (_, 0.18530689179...","[(anwhile, 1.03836617568492e-16), (icester, 3....","[(Ġexecute, 5.97450380155351e-05), (_, 0.18530..."
2,"def _decode(self, data):\n '''\n ...","[('def', 'def', 'function_definition'), ('_dec...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 12501, 1098, 7, 944, 11, 1366, 25...",583,"[(ers, 0.03083181567490101), ((, 0.01664732024...","[(anwhile, 1.03836617568492e-16), (ousy, 4.688...","[(Ġ_, 0.001834857277572155), (dec, 0.001334611..."
3,"def _repr_html_(self):\n """"""\n J...","[('def', 'def', 'function_definition'), ('_rep...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 260, 1050, 62, 6494, 41052, 944, ...",221,"[(ers, 0.03083181567490101), ((, 0.01664732024...","[(anwhile, 1.03836617568492e-16), (ousy, 4.688...","[(Ġ_, 0.001834857277572155), (re, 0.0013783742..."
4,"def build_shape(relation, nodes, ways):\n ""...","[('def', 'def', 'function_definition'), ('buil...","[(4299, 'def', 'function_definition'), (1382, ...","[4299, 1382, 62, 43358, 7, 49501, 11, 13760, 1...",454,"[(ers, 0.03083181567490101), (_, 0.28421667218...","[(anwhile, 1.03836617568492e-16), (buquerque, ...","[(Ġbuild, 0.0005856865900568664), (_, 0.284216..."


In [91]:
## Saving File
#data_pd.to_csv( 'output/testbed_base_EleutherAI-gpt-neo-125M.csv' )
dataframe_to_save.to_csv( 'output/testbed_base_EleutherAI-gpt-neo-125M.csv' )